In [1]:
import pandas as pd
import sqlalchemy
import os
from sqlalchemy import text
from dotenv import load_dotenv

# Load database credentials
load_dotenv()

DB_USER = os.getenv("DB_USER")
DB_PASS = os.getenv("DB_PASS")
DB_HOST = os.getenv("DB_HOST")
DB_PORT = os.getenv("DB_PORT")
DB_NAME = os.getenv("DB_NAME")

# Create SQLAlchemy engine
engine = sqlalchemy.create_engine(f"postgresql://{DB_USER}:{DB_PASS}@{DB_HOST}:{DB_PORT}/{DB_NAME}")

# Load data from RDS table
query = "SELECT * FROM sql_project.all_products_data;"
df = pd.read_sql(query, engine)

# Normalize column names
df.columns = df.columns.str.strip().str.lower().str.replace(" ", "_")

# Convert relevant fields to boolean
bool_cols = ['all_vegan', 'partial_vegan', 'bad_parent_company', 'black_owned']
for col in bool_cols:
    df[col] = df[col].astype(str).str.lower().map({'true': True, 'false': False})

# Group by retailer attributes
retailers = ['ulta', 'sephora', 'beauty_bay', 'cult_beauty', 'nordstrom']
grouped_data = {}

for retailer in retailers:
    counts = {
        'all_vegan': df[df[retailer] == 1]['all_vegan'].sum(),
        'partial_vegan': df[df[retailer] == 1]['partial_vegan'].sum(),
        'bad_parent_company': df[df[retailer] == 1]['bad_parent_company'].sum(),
        'black_owned': df[df[retailer] == 1]['black_owned'].sum(),
    }
    grouped_data[retailer] = counts

# Convert to DataFrame and reshape for one-line Looker Studio use
grouped_df = pd.DataFrame(grouped_data).T.reset_index()
grouped_df = grouped_df.rename(columns={'index': 'retailer'})
long_format_df = grouped_df.melt(id_vars='retailer', var_name='attribute', value_name='count')

# Save as one CSV file for Looker Studio
output_path = "../../Data/Retrieve/retailer_attributes_long_format.csv"
long_format_df.to_csv(output_path, index=False)

print(f"✅ Grouped data exported to: {output_path}")


✅ Grouped data exported to: ../../Data/Retrieve/retailer_attributes_long_format.csv
